In [ ]:
import mne
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Try to use interactive backend if available, otherwise inline
try:
    %matplotlib widget
except:
    %matplotlib inline

In [2]:
# --- 1. Setup Paths ---
# Current folder (assuming this notebook is in code/digitspan/)
scripts_folder = Path.cwd()
root_folder = scripts_folder.parent.parent
data_path = root_folder / 'data' / 'ds003838' / 'sub-032'
data_folder = data_path / 'ecg'

subject_id = 'sub-032'
task = 'rest'

print(f"📂 Data folder: {data_folder}")

📂 Data folder: /home/martin/RESEARCH/thesis/brain_heart_psv_sdg/data/ds003838/sub-032/ecg


In [3]:
# --- 2. Load Signal (.set) ---
set_filename = f"{subject_id}_task-{task}_ecg.set"
set_path = data_folder / set_filename

if not set_path.exists():
    raise FileNotFoundError(f"Could not find {set_path}")

print(f"⏳ Loading {set_filename}...")
# Load the binary file with MNE
raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
print(raw.info)

⏳ Loading sub-032_task-rest_ecg.set...


/home/martin/miniforge3/envs/neurocomp/lib/python3.12/site-packages/pymatreader/utils.py:179: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(


<Info | 8 non-empty values
 bads: []
 ch_names: PPG, ECG
 chs: 2 EEG
 custom_ref_applied: False
 dig: 2 items (2 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 2
 projs: []
 sfreq: 1000.0 Hz
>


/tmp/ipykernel_76095/1203144154.py:10: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
/tmp/ipykernel_76095/1203144154.py:10: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)


In [4]:
# --- 3. Load Events (.tsv) ---
tsv_filename = f"{subject_id}_task-{task}_events.tsv"
tsv_path = data_folder / tsv_filename

if tsv_path.exists():
    print(f"⏳ Loading events from {tsv_filename}...")
    events_df = pd.read_csv(tsv_path, sep='\t')
    print("Events found:")
    print(events_df.head())
    
    # Check for value or trial_type columns
    # Prefer 'value' as it often contains specific descriptions (e.g., 'eyes opened')
    if 'value' in events_df.columns:
        print(f"Event values: {events_df['value'].unique()}")
        descriptions = events_df['value'].values
    elif 'trial_type' in events_df.columns:
        print(f"Event types: {events_df['trial_type'].unique()}")
        descriptions = events_df['trial_type'].values
    else:
        print("⚠️ No 'value' or 'trial_type' column found in events.")
        descriptions = ['Event'] * len(events_df)
    
    # Create MNE Annotations
    # MNE Annotations need onset (seconds), duration (seconds), and description
    onsets = events_df['onset'].values
    durations = events_df['duration'].values
        
    annotations = mne.Annotations(onsets, durations, descriptions)
    raw.set_annotations(annotations)
    print("✅ Annotations added to raw data.")
else:
    print(f"⚠️ Events file not found: {tsv_filename}")

⏳ Loading events from sub-032_task-rest_events.tsv...
Events found:
      onset  duration    sample trial_type  response_time  stim_file  \
0   -0.0005   18441.0      -0.5     STATUS            NaN        NaN   
1  227.0790       1.0  227079.0     STATUS            NaN        NaN   

         value  
0     boundary  
1  eyes opened  
Event values: ['boundary' 'eyes opened']
✅ Annotations added to raw data.


/tmp/ipykernel_76095/2657081329.py:29: RuntimeWarning: Limited 2 annotation(s) that were expanding outside the data range.
  raw.set_annotations(annotations)


In [6]:
# --- 4. Visualize ---
# Plot the raw data with annotations
print("📊 Plotting data... ")
# Using 'scalings' to ensure ECG is visible. 'auto' usually works well.
raw.plot(duration=60, n_channels=len(raw.ch_names), scalings='auto', title=f"ECG - {subject_id} - {task}", show=True)

📊 Plotting data... 
